# DATASET LOADING

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/intras/images.hdf5
/kaggle/input/intras/data_only_numerical.csv
/kaggle/input/intras/subject_data.csv
/kaggle/input/model_for_preds/tensorflow2/default/1/best_combined_model_1.h5


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("/kaggle/input/intras/data_only_numerical.csv")
data.head()

,Unnamed: 0,target,age_approx,sex,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,iddx_full,iddx_1,tbp_lv_dnn_lesion_confidence
0,0,0,60.0,0,3.04,20.244422,16.261975,26.922447,23.954773,33.684638,...,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,0,0,97.517282
1,1,0,60.0,0,1.10,31.712570,25.364740,26.331000,24.549290,41.219030,...,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,0,0,3.141455
2,2,0,60.0,0,3.40,22.575830,17.128170,37.970460,33.485410,44.174920,...,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,0,0,99.804040
3,3,0,65.0,0,3.22,14.242329,12.164757,21.448144,21.121356,25.746200,...,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,0,0,99.989998
4,4,0,55.0,0,2.73,24.725520,20.057470,26.464900,25.710460,36.217980,...,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,0,0,70.442510


In [ ]:
data = data.drop(['iddx_full','iddx_1','tbp_lv_dnn_lesion_confidence','tbp_lv_nevi_confidence'],axis=1)

data.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
data_1 = pd.read_csv("/kaggle/input/intras/subject_data.csv")

/tmp/ipykernel_36/3951759125.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data_1 = pd.read_csv("/kaggle/input/intras/subject_data.csv")


In [ ]:
import h5py
import matplotlib.pyplot as plt
from PIL import Image
import io
import numpy as np
import cv2  # import OpenCV

file_path = "/kaggle/input/intras/images.hdf5"
isic_codes = data_1[' "isic_id"'].tolist()
images=[]
targets = data['target'].values
with h5py.File(file_path, "r") as hdf:
    ls = list(hdf.keys())
    for i in isic_codes:
        image_data = hdf[i]
        # Check if the dataset is scalar and contains bytes
        if image_data.shape == () and image_data.dtype.kind == 'S':
            # Read the scalar byte data
            image_bytes = image_data[()]
            img_pil = Image.open(io.BytesIO(image_bytes))
            images.append(img_pil)

        else:
            print(f"Item '{i}' is not a scalar byte string")

In [ ]:
data['images_idx'] = range(len(images))

mask = data['target'] == 1
df_ones = data[mask].copy()
df_zeros = data[~mask].copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train_ones, X_test_ones, y_train_ones, y_test_ones = train_test_split(df_ones.drop(columns=['target']), df_ones['target'], test_size=0.3, random_state=42)

In [ ]:
sampled_x_train_zeros, sampled_x_zeros, sampled_y_train_zeros, samples_y_zeros = train_test_split(df_zeros.drop(columns=['target']), df_zeros['target'], test_size=0.01, random_state=42)

In [ ]:
X_TR_ZEROS, X_TEST_ZEROS, Y_TR_ZEROS, Y_TEST_ZEROS = train_test_split(sampled_x_zeros, samples_y_zeros, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd

In [ ]:
X_train = pd.concat([X_train_ones, X_TR_ZEROS], ignore_index=True)
X_test  = pd.concat([X_test_ones, X_TEST_ZEROS], ignore_index=True)

# Labels (y) - keep as DataFrame, not array
y_train = pd.concat([y_train_ones, Y_TR_ZEROS], ignore_index=True)
y_test  = pd.concat([y_test_ones, Y_TEST_ZEROS], ignore_index=True)

In [ ]:
import torch as torch

In [ ]:
print(X_train.shape)
print(X_test.shape)

(3480, 35)
(920, 35)


In [ ]:
images_test = []
images_train = []
for i in X_test['images_idx'].to_numpy():
    images_test.append(images[i])
for i in X_train['images_idx'].to_numpy():
    images_train.append(images[i])

DATASET (WITH CENTRE ZOOM IN)

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor()
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pil_images, targets, X, transform=None):
        self.X = X
        self.images = pil_images
        self.targets = targets
        self.transform = transform
        assert len(self.images) == len(self.X) == len(self.targets), "Length mismatch"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, self.X.iloc[idx].values, label

CREATING DATALOADERS

In [ ]:
training = CustomDataset(images_train, y_train, X_train.drop(columns=['images_idx']), train_preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), test_preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle=True, pin_memory=True)

In [ ]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs.numpy())
        X.append(x.numpy())
        y.append(labels.numpy())
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_ = dataloader_to_numpy(test_dataloader)

import tensorflow as tf

seed_value = 42
tf.random.set_seed(seed_value)   # TensorFlow seed
np.random.seed(seed_value)       # NumPy seed

In [ ]:
print(len(training))

3480


In [ ]:
print(len(testing))

920


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Create the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath="best_combined_model.h5",  # save best combined model
    monitor="val_recall_with_threshold",               # monitor recall
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduce LR on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=10,
    verbose=1,
    min_lr=1e-5
)

# Fit combined model
history = model.fit(
    [X_train_img, X_train_],  # both image and numerical inputs
    y_train_,
    validation_data=([X_test_img, X_test_], y_test_),
    epochs=100,
    batch_size = 32,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0: 1, 1: 5},  # handle imbalance
    verbose=1
)

Epoch 1/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.5584 - precision: 0.0945 - recall_with_threshold: 0.7453
Epoch 1: val_recall_with_threshold improved from -inf to 0.87288, saving model to best_combined_model.h5
109/109 ━━━━━━━━━━━━━━━━━━━━ 54s 219ms/step - loss: 0.5573 - precision: 0.0946 - recall_with_threshold: 0.7447 - val_loss: 0.3001 - val_precision: 1.0000 - val_recall_with_threshold: 0.8729 - learning_rate: 0.0020
Epoch 2/100
108/109 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.3078 - precision: 0.1431 - recall_with_threshold: 0.3972
Epoch 2: val_recall_with_threshold did not improve from 0.87288
109/109 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 0.3076 - precision: 0.1431 - recall_with_threshold: 0.3967 - val_loss: 0.2552 - val_precision: 0.0000e+00 - val_recall_with_threshold: 0.2203 - learning_rate: 0.0020
Epoch 3/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.2568 - precision: 0.3039 - recall_with_threshold: 0.3576
Epoch 3: val_recall_with_threshold di

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
#model = tf.keras.models.load_model(
 #   '/kaggle/working/best_combined_model.h5',
 #   custom_objects={'binary_focal_loss': binary_focal_loss()},
 #   compile = False
#)

y_pred = model.predict([X_test_img, X_test_])
for threshold in np.linspace(0, 0.5, 50):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(f'roc_auc_score = {roc_auc_score(y_test_, y_)}')
    print(classification_report(y_test_, y_))

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step
threshold = 0.0
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.01020408163265306
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.02040816326530612
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.24489795918367346
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.25510204081632654
roc_auc_score = 0.5006234413965087
              precision    recall  f1-score   support

           0       1.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OVERSAMPLING OF TRAIN MINORITY CLASS BY OFFLINE AUGMENTATION

In [ ]:
# extracting positive sample images

ones_index = list(X_train_ones['images_idx'])
# print(ones_index)

images_train_ones  = [images[i] for i in ones_index]
# print(images_train_ones)

print(len(images_train_ones))

275


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

# Initialize your ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

augmented_images_train_ones = []
augmented_ones_train= []
k = 0
for pil_img in images_train_ones:
    x = img_to_array(pil_img)
    x = x.reshape((1,) + x.shape)  # reshape for datagen
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        augmented_images_train_ones.append(batch[0].astype(np.uint8))  # Store augmented image array
        i += 1
        augmented_ones_train.append(X_train_ones.iloc[k].values)
        if i >= 5:  # Number of augmentations per image
            break
    k+=1

2025-08-15 18:26:26.972021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755282387.159588      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755282387.209400      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
augmented_ones_df = pd.DataFrame(augmented_ones_train, columns = X_train_ones.columns)
X_train_new = pd.concat([X_train, augmented_ones_df], ignore_index=True)
images_train_new = images_train.copy()
augmented_images_train_ones_pil = [
    Image.fromarray(arr).convert('RGB') for arr in augmented_images_train_ones
]
for img in augmented_images_train_ones_pil:
    images_train_new.append(img)
len(images_train_new)

4855

In [ ]:
from tensorflow import keras
print(np.__version__)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

from torch.utils.data import DataLoader, Dataset

from torchvision import transforms

#check and adjust input dimensions
train_preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(int(0.75 * 128)),  # Crop 96x96
    transforms.Resize(128),
    transforms.ToTensor(),

])

test_preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor()
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pil_images, targets, X, transform=None):
        self.X = X
        self.images = pil_images
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, self.X.iloc[idx].values, label

In [ ]:
X_train_new.shape
y_train_new = [i for i in y_train]

for j in range(0, len(augmented_images_train_ones)):
    y_train_new.append(1)

y_train_new = np.array([val for val in y_train_new]).flatten()



In [ ]:
from torch.utils.data import DataLoader

In [ ]:
training = CustomDataset(images_train_new, y_train_new, X_train_new.drop(columns=['images_idx']), train_preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), test_preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle=True, pin_memory=True)

In [ ]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs.numpy())
        X.append(x.numpy())
        y.append(labels.numpy())
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_ = dataloader_to_numpy(test_dataloader)

RESNET (IMAGE BLOCK) + MLP (TABULAR DATA)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import (
    Input, Dense, Concatenate, BatchNormalization, Dropout, GlobalAveragePooling2D
)
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.initializers import HeNormal
import tensorflow as tf

# Custom Focal Loss and Custom Metric
def binary_focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * tf.pow(y_pred, gamma) * (1 - y_true)
        return tf.reduce_mean(weight * cross_entropy)
    return loss

class RecallWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.3, name="recall_with_threshold", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.recall.result()

    def reset_states(self):
        self.recall.reset_states()

# -------- Inputs --------
image_input = Input(shape=(128, 128, 3), name='image_input')
num_input = Input(shape=(34,), name='num_input')

# -------- Image branch with ResNet50 --------
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)
base_model.trainable = False  # freeze ResNet weights

x1 = base_model.output
x1 = GlobalAveragePooling2D()(x1)

x1 = Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)

x1 = Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.3)(x1)

x1 = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x1)

# -------- Numerical branch --------
x2 = BatchNormalization()(num_input)
x2 = Dense(48, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.2)(x2)

x2 = Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.2)(x2)

x2 = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x2)

# -------- Merge branches --------
merged = Concatenate()([x1, x2])
merged = BatchNormalization()(merged)
merged = Dropout(0.3)(merged)
merged = Dense(64, activation='relu')(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
merged = Dense(16, activation='elu')(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.1)(merged)
output = Dense(1, activation='sigmoid')(merged)

# -------- Build and Compile Model --------
model = Model(inputs=[image_input, num_input], outputs=output)

model.compile(
    optimizer=AdamW(learning_rate=2e-3),
    loss=binary_focal_loss(),
    metrics=['precision', RecallWithThreshold(0.3)]
)

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Create the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath="best_combined_model_7.h5",  # save best combined model
    monitor="val_recall_with_threshold",               # monitor recall
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduce LR on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=10,
    verbose=1,
    min_lr=1e-5
)

# Fit combined model
history = model.fit(
    [X_train_img, X_train_],  # both image and numerical inputs
    y_train_,
    validation_data=([X_test_img, X_test_], y_test_),
    epochs=100,
    batch_size = 32,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0: 1, 1: 5},  # to handle imbalance
    verbose=1
)

Epoch 1/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - loss: 0.7779 - precision: 0.4592 - recall_with_threshold: 0.6613
Epoch 1: val_recall_with_threshold improved from -inf to 0.44068, saving model to best_combined_model_7.h5
152/152 ━━━━━━━━━━━━━━━━━━━━ 172s 500ms/step - loss: 0.7770 - precision: 0.4598 - recall_with_threshold: 0.6614 - val_loss: 0.4015 - val_precision: 0.7368 - val_recall_with_threshold: 0.4407 - learning_rate: 0.0020
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.4979 - precision: 0.7419 - recall_with_threshold: 0.6534
Epoch 2: val_recall_with_threshold improved from 0.44068 to 0.61017, saving model to best_combined_model_7.h5
152/152 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - loss: 0.4977 - precision: 0.7422 - recall_with_threshold: 0.6536 - val_loss: 0.3335 - val_precision: 0.7581 - val_recall_with_threshold: 0.6102 - learning_rate: 0.0020
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.4085 - precision: 0.8163 - recall_with_threshold

KeyboardInterrupt: 

EXTRACTING THE FEATURES OF TRAIN INPUTS

In [ ]:
features = intermediate_model.predict([X_train_img, X_train_])
test_features = intermediate_model.predict([X_test_img, X_test_])

152/152 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 275ms/step


In [ ]:
import pandas as pd
import torch

ones_to_append = pd.Series([1] * 1375)
y_train = pd.concat([y_train, ones_to_append], ignore_index=True)


In [ ]:
import torch

#tensor of ones of length 1375, same dtype and device as y_train
ones_to_append = torch.ones(1375, dtype=y_train.dtype)

#concatenate y_train and the ones
y_train = torch.cat((y_train, ones_to_append))


APPLYING XGB ON RESNET EXTRACTED ARCHITECTURE

In [ ]:
import numpy as np
import xgboost as xgb


features = np.array(features)


y_train_np = np.array(y_train).ravel()
y_test_np = np.array(y_test).ravel()

print(features.shape)
print(y_train_np.shape)

dtrain = xgb.DMatrix(data=features, label=y_train_np)


(4855, 16)
(4855,)


USING XGB ON RESNET EXTRACTED FEATURES

In [ ]:
import xgboost as xgb

# Example: create DMatrix from extracted features and labels
dtrain = xgb.DMatrix(features, label=y_train)

# Define parameters and train an XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'seed': 42,
}

bst = xgb.train(params, dtrain, num_boost_round=100)

# For prediction on validation/test data (similarly extract features for X_test):
dtest = xgb.DMatrix(test_features,label = y_test_np)
y_pred = bst.predict(dtest)


In [ ]:
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Assume features, y_train_np are your training data and labels as NumPy arrays

# Create an XGBClassifier wrapper for scikit-learn compatibility
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss', seed=42)

# Define the hyperparameter search space
search_spaces = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3, 'log-uniform'),
    'n_estimators': (50, 200),
    'subsample': (0.6, 1.0, 'uniform'),
    'colsample_bytree': (0.6, 1.0, 'uniform'),
    'gamma': (0, 5),
    'min_child_weight': (1, 10)
}

# Set up Bayesian optimization with cross-validation
opt = BayesSearchCV(
    estimator=xgb_clf,
    search_spaces=search_spaces,
    scoring='roc_auc',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_iter=30,           # number of parameter settings sampled
    random_state=42,
    verbose=3,
    n_jobs=-1
)

# Fit Bayesian search on training data
opt.fit(features, y_train_np)

# Best hyperparameters and score
print("Best parameters found: ", opt.best_params_)
print("Best cross-validation ROC AUC: ", opt.best_score_)

# Predict on test set
dtest = xgb.DMatrix(test_features)
y_pred_prob = opt.predict_proba(test_features)[:, 1]

# Evaluate ROC AUC on test set if y_test_np is available
if 'y_test_np' in locals():
    test_roc_auc = roc_auc_score(y_test_np, y_pred_prob)
    print("Test ROC AUC:", test_roc_auc)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 4/5] END colsample_bytree=0.7640415835413256, gamma=4, learning_rate=0.2387586688716479, max_depth=5, min_child_weight=7, n_estimators=112, subsample=0.7403725339596576;, score=0.471 total time=   0.1s
[CV 1/5] END colsample_bytree=0.7640415835413256, gamma=4, learning_rate=0.2387586688716479, max_depth=5, min_child_weight=7, n_estimators=112, subsample=0.7403725339596576;, score=0.525 total time=   0.1s
[CV 2/5] END colsample_bytree=0.7640415835413256, gamma=4, learning_rate=0.2387586688716479, max_depth=5, min_child_weight=7, n_estimators=112, subsample=0.7403725339596576;, score=0.485 total time=   0.1s
[CV 3/5] END colsample_bytree=0.7640415835413256, gamma=4, learning_rate=0.2387586688716479, max_depth=5, min_child_weight=7, n_estimators=112, subsample=0.7403725339596576;, score=0.527 total time=   0.1s
[CV 5/5] END colsample_bytree=0.7640415835413256, gamma=4, learning_rate=0.2387586688716479, max_depth=5, min_child_w

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Predict using the XGBoost model 'bst' on the test DMatrix 'dtest'
y_pred_proba = bst.predict(dtest)

# Convert probabilities to binary predictions using a threshold, e.g., 0.5
y_pred = (y_pred_proba >= 0.25).astype(int)

# Assuming you have the true labels for test set as numpy array y_test_np
print("ROC AUC score:", roc_auc_score(y_test_np, y_pred_proba))
print(classification_report(y_test_np, y_pred))


ROC AUC score: 0.5393190751933725
              precision    recall  f1-score   support

           0       0.91      0.37      0.53       802
           1       0.15      0.74      0.25       118

    accuracy                           0.42       920
   macro avg       0.53      0.55      0.39       920
weighted avg       0.81      0.42      0.49       920



RESULTS OF RESNET

In [ ]:
model_new = tf.keras.models.load_model(
    '/kaggle/working/best_combined_model_6.h5',
    custom_objects={'binary_focal_loss': binary_focal_loss()},
    compile = False
)
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model_new.predict([X_test_img, X_test_])
for threshold in np.linspace(0, 0.5, 50):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(f'roc_auc_score = {roc_auc_score(y_test_, y_)}')
    print(classification_report(y_test_, y_))

29/29 ━━━━━━━━━━━━━━━━━━━━ 30s 565ms/step
threshold = 0.0
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.01020408163265306
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.02040816326530612
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.98      0.76      0.86       802
           1       0.35      0.88      0.50       118

    accuracy                           0.78       920
   macro avg       0.67      0.82      0.68       920
weighted avg       0.90      0.78      0.81       920

threshold = 0.3061224489795918
roc_auc_score = 0.8206073798554461
              precision    recall  f1-score   support

           0       0.97      0.78      0.86       802
           1       0.36      0.86      0.51       118

    accuracy                           0.79       920
   macro avg       0.67      0.82      0.69       920
weighted avg       0.90      0.79      0.82       920

threshold = 0.31632653061224486
roc_auc_score = 0.8183672175493469
              precision    recall  f1-score   support

           0       0.97      0.79      0.87       802
           1       0.37      0.85      0.52       118

    accuracy                           0.80      

--------------------------------------------------------------------------------

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_selected_num_zeros, y_selected_zeros,test_size=0.2,random_state=42, stratify = y_selected_zeros)

In [ ]:
print(len(X_train_ones))

275


In [ ]:
X_train = pd.concat([X_train,X_train_ones],ignore_index=True)
y_train = pd.concat([y_train, y_train_ones],ignore_index=True)

X_test = pd.concat([X_test, X_test_ones],ignore_index=True)
y_test = pd.concat([y_test, y_test_ones],ignore_index=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_selected_num, y_selected,test_size=0.2,random_state=42, stratify = y_selected)

In [ ]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [ ]:
images_test = []
images_train = []
for i in X_test['images_idx'].to_numpy():
    images_test.append(images[i])
for i in X_train['images_idx'].to_numpy():
    images_train.append(images[i])

In [ ]:
print(len(images_train))
print(len(images_test))

2152
519


In [ ]:
import torch

# Your existing tensor
# y_train is a 1D tensor, e.g. shape (N,)
# Create a tensor of ones with length 275, of same type and device as y_train
ones_to_append = torch.ones(275, dtype=y_train.dtype, device=y_train.device)

# Concatenate tensors along dimension 0 (append to the end)
y_train = torch.cat((y_train, ones_to_append))


In [ ]:
print(len(y_train))
print(len(y_test))

2152
519


In [ ]:
training = CustomDataset(images_train, y_train, X_train.drop(columns=['images_idx']), train_preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), test_preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle=True, pin_memory=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *

In [ ]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs.numpy())
        X.append(x.numpy())
        y.append(labels.numpy())
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_ = dataloader_to_numpy(test_dataloader)

import tensorflow as tf

seed_value = 42
tf.random.set_seed(seed_value)   # TensorFlow seed
np.random.seed(seed_value)       # NumPy seed

In [ ]:
import tensorflow as tf

class RecallWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.5, name='recall_with_threshold', **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred >= self.threshold, tf.float32)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.recall.result()

    def reset_state(self):
        self.recall.reset_state()

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Create the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath="best_combined_model_6.h5",  # save best combined model
    monitor="val_recall_with_threshold",               # monitor recall
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduce LR on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=10,
    verbose=1,
    min_lr=1e-5
)

# Fit combined model
history = model.fit(
    [X_train_img, X_train_],  # both image and numerical inputs
    y_train_,
    validation_data=([X_test_img, X_test_], y_test_),
    epochs=100,
    batch_size = 32,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0: 1, 1: 3},  # handle imbalance
    verbose=1
)

Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 0.5866 - precision: 0.3065 - recall_with_threshold: 0.8404
Epoch 1: val_recall_with_threshold improved from -inf to 1.00000, saving model to best_combined_model_6.h5
68/68 ━━━━━━━━━━━━━━━━━━━━ 52s 308ms/step - loss: 0.5852 - precision: 0.3071 - recall_with_threshold: 0.8406 - val_loss: 0.4432 - val_precision: 0.2397 - val_recall_with_threshold: 1.0000 - learning_rate: 0.0020
Epoch 2/100
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.3603 - precision: 0.5501 - recall_with_threshold: 0.8351
Epoch 2: val_recall_with_threshold did not improve from 1.00000
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.3600 - precision: 0.5519 - recall_with_threshold: 0.8344 - val_loss: 0.3124 - val_precision: 0.7111 - val_recall_with_threshold: 0.9068 - learning_rate: 0.0020
Epoch 3/100
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.3153 - precision: 0.8204 - recall_with_threshold: 0.7573
Epoch 3: val_recall_with_threshold did not improv

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Create the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath="best_combined_model_3.h5",  # save best combined model
    monitor="val_recall_with_threshold",               # monitor recall
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduce LR on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=10,
    verbose=1,
    min_lr=1e-5
)

# Fit combined model
history = model.fit(
    [X_train_img, X_train_],  # both image and numerical inputs
    y_train_,
    validation_data=([X_test_img, X_test_], y_test_),
    epochs=100,
    batch_size = 32,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0: 1, 1: 3},  # handle imbalance
    verbose=1
)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - loss: 0.6977 - precision: 0.2043 - recall_with_threshold: 0.7897
Epoch 1: val_recall_with_threshold improved from -inf to 1.00000, saving model to best_combined_model_3.h5
25/25 ━━━━━━━━━━━━━━━━━━━━ 51s 892ms/step - loss: 0.6946 - precision: 0.2042 - recall_with_threshold: 0.7890 - val_loss: 0.8041 - val_precision: 0.1950 - val_recall_with_threshold: 1.0000 - learning_rate: 0.0020
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.4865 - precision: 0.2746 - recall_with_threshold: 0.8561
Epoch 2: val_recall_with_threshold did not improve from 1.00000
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.4857 - precision: 0.2729 - recall_with_threshold: 0.8550 - val_loss: 0.3438 - val_precision: 0.3000 - val_recall_with_threshold: 0.8974 - learning_rate: 0.0020
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.4069 - precision: 0.2822 - recall_with_threshold: 0.7430
Epoch 3: val_recall_with_threshold did not improv

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
model = tf.keras.models.load_model(
    '/kaggle/working/best_combined_model_3.h5',
    custom_objects={'binary_focal_loss': binary_focal_loss()},
    compile = False
)

y_pred = model.predict([X_test_img, X_test_])
for threshold in np.linspace(0, 0.5, 50):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(f'roc_auc_score = {roc_auc_score(y_test_, y_)}')
    print(classification_report(y_test_, y_))

7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 703ms/step
threshold = 0.0
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       161
         1.0       0.20      1.00      0.33        39

    accuracy                           0.20       200
   macro avg       0.10      0.50      0.16       200
weighted avg       0.04      0.20      0.06       200

threshold = 0.01020408163265306
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       161
         1.0       0.20      1.00      0.33        39

    accuracy                           0.20       200
   macro avg       0.10      0.50      0.16       200
weighted avg       0.04      0.20      0.06       200

threshold = 0.02040816326530612
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       161
         1.0       0.20      1.00      0.33        39

   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       161
         1.0       0.20      1.00      0.33        39

    accuracy                           0.20       200
   macro avg       0.10      0.50      0.16       200
weighted avg       0.04      0.20      0.06       200

threshold = 0.3469387755102041
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       161
         1.0       0.20      1.00      0.33        39

    accuracy                           0.20       200
   macro avg       0.10      0.50      0.16       200
weighted avg       0.04      0.20      0.06       200

threshold = 0.3571428571428571
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       161
         1.0       0.20      1.00      0.33        39

    accuracy                           0.20       200
   macro avg       0.10   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

In [ ]:
class RecallWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.3, name="recall_with_threshold", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.recall.result()

    def reset_states(self):
        self.recall.reset_states()

EFFICIENT NET MODEL ARCHITECTURE

In [ ]:
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.layers import (
    Input, Dense, Concatenate, BatchNormalization, Dropout, GlobalAveragePooling2D
)
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.initializers import HeNormal
import tensorflow as tf

# Custom Focal Loss and Custom Metric
def binary_focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * tf.pow(y_pred, gamma) * (1 - y_true)
        return tf.reduce_mean(weight * cross_entropy)
    return loss

class RecallWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.3, name="recall_with_threshold", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.recall.result()

    def reset_states(self):
        self.recall.reset_states()

# -------- Inputs --------
image_input = Input(shape=(128, 128, 3), name='image_input')
num_input = Input(shape=(34,), name='num_input')

# -------- Image branch with EfficientNetV2-M --------
base_model = EfficientNetV2M(weights='imagenet', include_top=False, input_tensor=image_input)
base_model.trainable = False  # freeze EfficientNet weights

x1 = base_model.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)
x1 = Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.3)(x1)
x1 = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x1)

# -------- Numerical branch --------
x2 = BatchNormalization()(num_input)
x2 = Dense(48, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.2)(x2)
x2 = Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.2)(x2)
x2 = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3))(x2)

# -------- Merge branches --------
merged = Concatenate()([x1, x2])
merged = BatchNormalization()(merged)
merged = Dropout(0.3)(merged)
merged = Dense(64, activation='relu')(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
merged = Dense(16, activation='elu')(merged)
merged = BatchNormalization(name='merged_bn')(merged)
merged = Dropout(0.1)(merged)
output = Dense(1, activation='sigmoid')(merged)



# -------- Build and Compile Model --------
model = Model(inputs=[image_input, num_input], outputs=output)
model.compile(
    optimizer=AdamW(learning_rate=2e-3),
    loss=binary_focal_loss(),
    metrics=['precision', RecallWithThreshold(0.35)]
)
model.summary()

intermediate_model = Model(inputs=model.inputs, outputs=model.get_layer('merged_bn').output)

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_7         │ (None, 128, 128,  │          0 │ image_input[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 64, 64,    │        648 │ rescaling_7[0][0] │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 64, 64,    │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 64, 64,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 64, 64,    │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 64, 64,    │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 64, 64,    │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 64, 64,    │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 64, 64,    │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 64, 64,    │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 64, 64,    │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 64, 64,    │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 64, 64,    │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_co… │ (None, 64, 64,    │      5,184 │ block1b_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_bn  │ (None, 64, 64,    │         96 │ block1c_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_ac… │ (None, 64, 64,    │          0 │ block1c_project_

 Total params: 53,243,213 (203.11 MB)

 Trainable params: 92,181 (360.08 KB)

 Non-trainable params: 53,151,032 (202.76 MB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Create the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath="EFFICIENT_NET.h5",  # save best combined model
    monitor="val_recall_with_threshold",               # monitor recall
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduce LR on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=10,
    verbose=1,
    min_lr=1e-5
)

# Fit combined model
history = model.fit(
    [X_train_img, X_train_],  # both image and numerical inputs
    y_train_,
    validation_data=([X_test_img, X_test_], y_test_),
    epochs=100,
    batch_size = 32,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0: 1, 1: 5},  # handle imbalance
    verbose=1
)

Epoch 1/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - loss: 0.7099 - precision: 0.4501 - recall_with_threshold: 0.7265
Epoch 1: val_recall_with_threshold improved from -inf to 0.51695, saving model to best_combined_model_4.h5
152/152 ━━━━━━━━━━━━━━━━━━━━ 176s 502ms/step - loss: 0.7093 - precision: 0.4505 - recall_with_threshold: 0.7263 - val_loss: 0.3960 - val_precision: 0.7381 - val_recall_with_threshold: 0.5169 - learning_rate: 0.0020
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.4794 - precision: 0.7458 - recall_with_threshold: 0.6797
Epoch 2: val_recall_with_threshold improved from 0.51695 to 0.65254, saving model to best_combined_model_4.h5
152/152 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - loss: 0.4792 - precision: 0.7458 - recall_with_threshold: 0.6797 - val_loss: 0.3202 - val_precision: 0.7200 - val_recall_with_threshold: 0.6525 - learning_rate: 0.0020
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.3923 - precision: 0.8157 - recall_with_threshold

KeyboardInterrupt: 

EFFICIENT NET METRICS

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
model = tf.keras.models.load_model(
    '/kaggle/working/EFFICIENT_NET.h5',
    custom_objects={'binary_focal_loss': binary_focal_loss()},
    compile = False
)

y_pred = model.predict([X_test_img, X_test_])
for threshold in np.linspace(0, 0.5, 50):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(f'roc_auc_score = {roc_auc_score(y_test_, y_)}')
    print(classification_report(y_test_, y_))

29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 591ms/step
threshold = 0.0
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.01020408163265306
roc_auc_score = 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       802
           1       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.02040816326530612
roc_auc_score = 0.5006234413965087
              precision    recall  f1-score   support

           0       1.00      0.00      0.00       802
           1       0.13      1.00      0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.98      0.79      0.87       802
           1       0.38      0.87      0.53       118

    accuracy                           0.80       920
   macro avg       0.68      0.83      0.70       920
weighted avg       0.90      0.80      0.83       920

threshold = 0.29591836734693877
roc_auc_score = 0.8343230905786381
              precision    recall  f1-score   support

           0       0.98      0.80      0.88       802
           1       0.39      0.86      0.54       118

    accuracy                           0.81       920
   macro avg       0.68      0.83      0.71       920
weighted avg       0.90      0.81      0.84       920

threshold = 0.3061224489795918
roc_auc_score = 0.8386871803541992
              precision    recall  f1-score   support

           0       0.98      0.81      0.89       802
           1       0.40      0.86      0.55       118

    accuracy                           0.82      